# Variable Lens Sequence

A system with multiple lenses in sequence. The number of lenses of each type is an input parameter of the script:

* Number of plano convex lenses
* Number of biconvex lenses
* Number of reverse plano

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

import math
import numpy as np

### DESIGN ###

# Design parameters
square_size = 30
lens_diameter = math.sqrt(2)*square_size
focal_length = 35.

# Number of each lens type
nplano = 1
nbiconvex = 2
nrplano = 1

# Mechanical sizes
lens_min_thickness = 1.2
lens_spacing = 3.

### MODEL ###

# Parametric surfaces 
surface_plano = tlm.Parabola(lens_diameter, tlm.parameter(-0.002))
surface_biconvex = tlm.Parabola(lens_diameter, tlm.parameter(0.002))

# Lenses
lens_plano = tlm.PlanoLens(
    surface_plano,
    n = (1.0, 1.5),
    outer_thickness = lens_min_thickness,
    reverse=False,
)

lens_biconvex = tlm.BiLens(
    surface_biconvex,
    n = (1.0, 1.5),
    outer_thickness = lens_min_thickness,
)
    
lens_rplano = tlm.PlanoLens(
    surface_plano,
    n = (1.0, 1.5),
    outer_thickness = lens_min_thickness,
    reverse=True,
)

optics = nn.Sequential(
    tlm.PointSourceAtInfinity(beam_diameter=0.9*lens_diameter),
    tlm.Gap(10.),
    *[
        tlm.Gap(lens_spacing),
        lens_plano,
    ]*nplano,
    *[
        tlm.Gap(lens_spacing),
        lens_biconvex,
    ] *nbiconvex,
    *[
        tlm.Gap(lens_spacing),
        lens_rplano,
    ]*nrplano,
    tlm.Gap(focal_length),
    tlm.FocalPoint(),
)

print("Lens design")
print("Square size", square_size)
print("Lens diameter", lens_diameter)
print("Configuration", nplano, nbiconvex, nrplano)
print("lens_min_thickness", lens_min_thickness)
print("lens_spacing", lens_spacing)

tlm.show(optics, dim=2)
tlm.show(optics, dim=3)

In [ ]:
def regu_equalparam(_):
    a1 = surface_plano.a
    a2 = surface_biconvex.a
    return 500*torch.pow((500*a1)**2 - (500*a2)**2, 2)
    #params = torch.cat([param.view(-1) for param in optics.parameters()])
    #return torch.pow(torch.diff(1000*torch.abs(params)).sum(), 2)

def regu_equalthickness(_):
    t0 = lens_plano.inner_thickness()
    t1 = lens_biconvex.inner_thickness()
    return 100*torch.pow(t0 - t1, 2)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-4),
    sampling = {"base": 10},
    dim = 2,
    num_iter = 200
).plot()

def print_lens(lens_name, lens):
    # TODO thickness at a specific radial distance
    print(lens_name)
    inner = lens.inner_thickness().item()
    outer = lens.outer_thickness().item()
    print(f"    inner: {inner:.3f} outer: {outer:.3f}")
    
    a1 = lens.surface1.surface.parameters()
    a2 = lens.surface2.surface.parameters()
    print("    surface1", [p.tolist() for p in a1.values()])
    print("    surface2", [p.tolist() for p in a2.values()])

print_lens("Plano-convex", lens_plano)
print_lens("Bi-convex", lens_biconvex)
print_lens("Reverse plano-convex", lens_rplano)


In [ ]:
tlm.show(optics, dim=2)
tlm.show(optics, dim=3)

In [ ]:
from IPython.display import display

display(tlm.export.lens_to_part(lens_plano))
display(tlm.export.lens_to_part(lens_biconvex))